In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from osgeo import gdal, ogr
import os

In [2]:
feature_list = ['PT_NDMI_0','PT_NDMI_1','PT_NDMI_2','PT_NDMI_3','PT_NDMI_4','PT_NDMI_5','PR_NDMI_1','PR_NDMI_2','PT_NBR_0',\
            'PT_NBR_1','PT_NBR_2','PT_NBR_3','PT_NBR_4','PT_NBR_5','PR_NBR_1','PR_NBR_2','PT_NDVI_0','PT_NDVI_1',\
            'PT_NDVI_2','PT_NDVI_3','PT_NDVI_4','PT_NDVI_5','TXU_ENT','TXU_asm','area','LT_mag',\
            'LT_dsnr','LT_dur','TXU_HOM','TXU_CON','Perimeter','D/A','Perimeter','Shape Index''NDVI_rvalue','NDMI_rvalue',
                'NBR_rvalue','NDVI_slope','NDMI_slope','NBR_slope']
list_first = ['PT_NDMI_0','PT_NDMI_1','PT_NDMI_2','PT_NDMI_3','PT_NDMI_4','PT_NDMI_5','PR_NDMI_1','PR_NDMI_2','PT_NBR_0',\
            'PT_NBR_1','PT_NBR_2','PT_NBR_3','PT_NBR_4','PT_NBR_5','PR_NBR_1','PR_NBR_2','PT_NDVI_0','PT_NDVI_1',\
            'PT_NDVI_2','PT_NDVI_3','PT_NDVI_4','PT_NDVI_5','TXU_ENT','TXU_asm','area','LT_mag',\
            'LT_dsnr','LT_dur','TXU_HOM','TXU_CON','Perimeter','D/A','Perimeter','Shape Index']

In [3]:
# canad_model = joblib.load("canada.model")
ne_model = joblib.load("ne_shpindex.model")
driver = ogr.GetDriverByName('ESRI Shapefile')
path = 'D:\\yyx\\GEE\\data\\shp_index\\'

In [4]:
def add_field_prediction(path):
    shp = driver.Open(path, 1)
    layer = shp.GetLayer()
#     field_canda = ogr.FieldDefn('type_cd', ogr.OFTReal)
    field_ne = ogr.FieldDefn('type_ne3', ogr.OFTReal)
#     layer.CreateField(field_canda)
    layer.CreateField(field_ne)
    for i in range(len(layer)):
        feature = layer.GetFeature(i)
        input_data = pd.DataFrame(columns=list_first)
        input_data = input_data.append(feature.items(), ignore_index=True)
        ts = pd.Series(input_data[list_first].values[0])
        if ts.isnull().any():
            feature.SetField("type_ne3", 3)
            continue
        ndvi = get_slope('NDVI', input_data)
        ndmi = get_slope('NDMI', input_data)
        nbr = get_slope('NBR', input_data)
        input_data = input_data.join(ndvi)
        input_data = input_data.join(ndmi)
        input_data = input_data.join(nbr)
        input_data = input_data[feature_list].values
#         prediction_canada = canad_model.predict(input_data)
        prediction_ne = ne_model.predict(input_data)
        feature.SetField("type_ne3", prediction_ne[0])
#         feature.SetField("type_cd", prediction_canada[0])
        layer.SetFeature(feature)
    shp.Destroy()
    print(path)
    
    
def add_index(name):
    index_list = []
    for i in range(1, 6):
        index_list.append('PT_' + name + '_' + str(i))
    return index_list


def get_slope(index, data):
    feature_list = [index + '_slope', index + '_intercept', index + '_rvalue', index + '_pvalue']
    dataframe = pd.DataFrame(columns=feature_list)
    index_list = add_index(index)
    x = [1, 2, 3, 4, 5]
    y_index = data[index_list].values
    for row in y_index:
        regress = stats.linregress(x, row)
        regress = [regress.slope, regress.intercept, regress.rvalue, regress.pvalue]
        data_regress = pd.Series(dict(zip(feature_list,regress)))
        dataframe = dataframe.append(data_regress, ignore_index=True)
    return dataframe

In [5]:
for year in range(1990, 2015):
    for i in range(115, 137):
        for j in range(54, 37, -1):
#             input_path = path + str(year) + '_' + str(i) + '_' + str(j) + '.shp'
#             if os.path.exists(input_path) is True:
#                 add_field_prediction(input_path)
            for k in range(0, 16):
                input_path = path + str(year) + '\\' + str(year) + '_' + str(i) + '_' + str(j) + '_' + str(k) + '.shp'
                if os.path.exists(input_path):
                    add_field_prediction(input_path)

D:\yyx\GEE\data\shp_index\1997//1997_123_49_12.shp
D:\yyx\GEE\data\shp_index\1997//1997_123_49_13.shp
D:\yyx\GEE\data\shp_index\1997//1997_123_49_14.shp
D:\yyx\GEE\data\shp_index\1997//1997_123_49_15.shp
D:\yyx\GEE\data\shp_index\1999//1999_123_49_12.shp
D:\yyx\GEE\data\shp_index\1999//1999_123_49_13.shp
D:\yyx\GEE\data\shp_index\1999//1999_123_49_14.shp
D:\yyx\GEE\data\shp_index\1999//1999_123_49_15.shp
D:\yyx\GEE\data\shp_index\1999//1999_124_50_7.shp
D:\yyx\GEE\data\shp_index\1999//1999_124_50_10.shp
D:\yyx\GEE\data\shp_index\1999//1999_124_50_11.shp
D:\yyx\GEE\data\shp_index\1999//1999_124_50_14.shp
D:\yyx\GEE\data\shp_index\1999//1999_124_50_15.shp
D:\yyx\GEE\data\shp_index\1999//1999_124_49_2.shp
D:\yyx\GEE\data\shp_index\1999//1999_124_49_3.shp
D:\yyx\GEE\data\shp_index\1999//1999_124_49_5.shp
D:\yyx\GEE\data\shp_index\1999//1999_124_49_6.shp
D:\yyx\GEE\data\shp_index\1999//1999_124_49_7.shp
D:\yyx\GEE\data\shp_index\1999//1999_124_49_9.shp
D:\yyx\GEE\data\shp_index\1999//1999_1

D:\yyx\GEE\data\shp_index\2003//2003_116_54_14.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_54_15.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_0.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_1.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_2.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_3.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_4.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_5.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_6.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_7.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_8.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_9.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_10.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_11.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_12.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_13.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_14.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_53_15.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_52_0.shp
D:\yyx\GEE\data\shp_index\2003//2003_116_5

D:\yyx\GEE\data\shp_index\2003//2003_121_52_7.shp
D:\yyx\GEE\data\shp_index\2003//2003_121_52_8.shp
D:\yyx\GEE\data\shp_index\2003//2003_121_52_9.shp
D:\yyx\GEE\data\shp_index\2003//2003_121_52_10.shp
D:\yyx\GEE\data\shp_index\2003//2003_121_52_11.shp
D:\yyx\GEE\data\shp_index\2003//2003_121_52_12.shp
D:\yyx\GEE\data\shp_index\2003//2003_121_52_13.shp
D:\yyx\GEE\data\shp_index\2003//2003_121_52_14.shp
D:\yyx\GEE\data\shp_index\2003//2003_121_52_15.shp
D:\yyx\GEE\data\shp_index\2003//2003_122_54_0.shp
D:\yyx\GEE\data\shp_index\2003//2003_122_54_1.shp
D:\yyx\GEE\data\shp_index\2003//2003_122_54_2.shp
D:\yyx\GEE\data\shp_index\2003//2003_122_54_3.shp
D:\yyx\GEE\data\shp_index\2003//2003_122_54_4.shp
D:\yyx\GEE\data\shp_index\2003//2003_122_54_5.shp
D:\yyx\GEE\data\shp_index\2003//2003_122_54_6.shp
D:\yyx\GEE\data\shp_index\2003//2003_122_54_7.shp
D:\yyx\GEE\data\shp_index\2003//2003_122_54_8.shp
D:\yyx\GEE\data\shp_index\2003//2003_122_54_9.shp
D:\yyx\GEE\data\shp_index\2003//2003_122_54_

D:\yyx\GEE\data\shp_index\2008//2008_132_52_1.shp
D:\yyx\GEE\data\shp_index\2008//2008_132_52_2.shp
D:\yyx\GEE\data\shp_index\2008//2008_132_52_3.shp
D:\yyx\GEE\data\shp_index\2008//2008_132_52_4.shp
D:\yyx\GEE\data\shp_index\2008//2008_132_52_5.shp
D:\yyx\GEE\data\shp_index\2008//2008_132_52_6.shp
D:\yyx\GEE\data\shp_index\2008//2008_132_52_7.shp
D:\yyx\GEE\data\shp_index\2008//2008_132_52_8.shp
D:\yyx\GEE\data\shp_index\2008//2008_132_52_9.shp
D:\yyx\GEE\data\shp_index\2008//2008_132_52_10.shp
D:\yyx\GEE\data\shp_index\2008//2008_132_52_11.shp
D:\yyx\GEE\data\shp_index\2008//2008_132_52_12.shp
D:\yyx\GEE\data\shp_index\2008//2008_132_52_13.shp
D:\yyx\GEE\data\shp_index\2008//2008_132_52_14.shp
D:\yyx\GEE\data\shp_index\2008//2008_132_52_15.shp
D:\yyx\GEE\data\shp_index\2009//2009_119_45_0.shp
D:\yyx\GEE\data\shp_index\2009//2009_119_45_1.shp
D:\yyx\GEE\data\shp_index\2009//2009_119_45_2.shp
D:\yyx\GEE\data\shp_index\2009//2009_119_45_3.shp
D:\yyx\GEE\data\shp_index\2009//2009_119_45_